In [72]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import wave

# things to do
 # lector de waves a SIGNAL
 # Agregar funcion para agregar o un SIN u otra SIGNAL A un SIN o SIGNAL
 # Agregar la funcionalidad para elegir el tipo de cambio cuando se hace una concatenacion de sonido
 # agregar las demas cosas que se pueden agregar a un sintetizador
 # agregar el manejador de notas acordes, progresiones y demas cosas
 # ver una manera de automatizar el hecho de obtener samples



def waveT(wave_file):
    # Abre el archivo WAV en modo de lectura
    with wave.open(wave_file, 'rb') as wav_file:
        n_channels = wav_file.getnchannels()  # Número de canales
        sample_width = wav_file.getsampwidth()  # Ancho de muestra en bytes
        framerate = wav_file.getframerate()  # Tasa de muestreo (frames por segundo)
        n_frames = wav_file.getnframes()  # Número total de frames

        # Lee los datos del archivo WAV
        audio_data = wav_file.readframes(n_frames)

        # Convierte los datos binarios a un array de numpy según el tipo de dato
        if sample_width == 1:
            # 8-bit WAV files are unsigned
            data = np.frombuffer(audio_data, dtype=np.uint8) - 128
        elif sample_width == 2:
            # 16-bit WAV files are signed
            data = np.frombuffer(audio_data, dtype=np.int16)
        elif sample_width == 4:
            # 32-bit WAV files are signed
            data = np.frombuffer(audio_data, dtype=np.int32)

        # Normaliza los datos para que estén en el rango [-1, 1] (esto es común para la reproducción de audio)
        max_range = float(2 ** (8 * sample_width - 1))
        data = data / max_range

        # Si es estéreo, convierte a mono para simplificar
        if n_channels == 2:
            data = data.reshape((-1, 2)).mean(axis=1)

    # Reproduce el audio
    return Audio(data, rate=framerate)


class SIGNAL:
    def __init__(self, data, time, fm=44100):
        self.data = data
        self.t = time
        self.fm = fm

    def show(self):
        plt.plot(self.t, self.data)
        plt.show()

    def get_audio(self):
        return Audio(self.data, rate=self.fm)

    def __add__(self, other):
        if len(self.t) == len(other.t):
            return SIGNAL(self.data + other.data, self.t, self.fm)
        else:
            print("Error: Time vectors do not match.")

    def __or__(self, other):
        # Calculate the time increment based on the time vector of the first signal
        time_step = np.diff(self.t[:2])[0]

        # Create the new time vector for the second signal starting right after the last time of the first
        shifted_t = other.t + self.t[-1] + time_step

        # Concatenate the original and the shifted time vectors
        new_t = np.concatenate((self.t, shifted_t))

        # Concatenate the data arrays
        new_data = np.concatenate((self.data, other.data))

        return SIGNAL(new_data, new_t, self.fm)

class SIN:
    def __init__(self, amplitud, frecuencia, desfase, tiempo, frecuencia_muestreo = 44100):
        self.amplitud = amplitud
        self.frecuencia = frecuencia
        self.desfase = desfase
        self.tiempo = tiempo
        self.frecuencia_muestreo = frecuencia_muestreo
        self.t = np.arange(int(frecuencia_muestreo * tiempo)) / frecuencia_muestreo
        self.data = self.amplitud * np.sin((2 * np.pi * self.frecuencia * self.t) + self.desfase)

    def show(self):
        plt.plot(self.t, self.data)
        plt.show()

    def get_signal(self):
        return self.data

    def get_time_vector(self):
        return self.t

    def __add__(self, other):
        if len(self.t) == len(other.t):
            return SIGNAL(self.data + other.data, self.t, self.frecuencia_muestreo)
        else:
            print("Error: Time vectors do not match.")

    def __str__(self):
        return f"{self.data}"

    def get_audio(self):
        return Audio(self.data, rate=self.frecuencia_muestreo)


In [70]:
do = SIN(10.0,261.63,0,1,44100)
do2 = SIN(10.0, 261.63*2,0,1,44100)
do3 = SIN(10.0, 261.63*3,0,1,44100)
do4 = SIN(10.0, 261.63*4,0,1,44100)
re = SIN(10.0,293.66,0,1,44100)
mi = SIN(10.0,329.63,0,1,44100)
fa = SIN(10.0,349.23,0,1,44100)
sol = SIN(10.0,392.0,0,1,44100)
E = SIN(10,659.29,0,1,44100)
prueba = (do+ do3)|(mi+sol)|(do4+ sol)
back = SIN(10,23.0,0,3,44100)
back2 = SIN(10,47,0,3,44100)
b1 = back + back2
prueba.get_audio()
print((do.get_time_vector()).size,E.get_time_vector().size,prueba.t.size)

44100 44100 132300


In [64]:
back.get_audio()

In [65]:
back2.get_audio()

In [69]:
j = back2 + back
j.get_audio()
b = prueba + j
b.get_audio()

In [74]:
readWave("test.wav")

In [ ]:
!pip install simpleaudio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import simpleaudio as sa

class SIGNAL:
    def __init__(self, data, time, fm=44100):
        self.data = data
        self.t = time
        self.fm = fm

    def show(self):
        plt.plot(self.t, self.data)
        plt.show()

    def play_audio(self):
        audio_data = (self.data * 32767).astype(np.int16)  # Convertir a formato WAV (16 bits)
        play_obj = sa.play_buffer(audio_data, 1, 2, self.fm)
        play_obj.wait_done()

class SIN:
    def __init__(self, amplitud, frecuencia, desfase, tiempo, frecuencia_muestreo):
        self.amplitud = amplitud
        self.frecuencia = frecuencia
        self.desfase = desfase
        self.tiempo = tiempo
        self.frecuencia_muestreo = frecuencia_muestreo
        self.t = np.arange(int(self.tiempo * self.frecuencia_muestreo)) / self.frecuencia_muestreo
        self.data = self.amplitud * np.sin(2 * np.pi * self.frecuencia * self.t + self.desfase)

    def show(self):
        plt.plot(self.get_time_vector(), self.get_signal())
        plt.show()

    def get_signal(self):
        return self.data

    def get_time_vector(self):
        return self.t

    def __add__(self, other):
        if len(self.get_time_vector()) == len(other.get_time_vector()):
            return SIGNAL(self.get_signal() + other.get_signal(), self.get_time_vector(), self.frecuencia_muestreo)
        else:
            print("Aún no resuelvo eso")

    def __str__(self):
        return f"{self.get_signal()}"

    def play_audio(self):
        audio_data = (self.get_signal() * 32767).astype(np.int16)  # Convertir a formato WAV (16 bits)
        play_obj = sa.play_buffer(audio_data, 1, 2, self.frecuencia_muestreo)
        play_obj.wait_done()

In [ ]:
test = SIN(10,389.0,-5.0,1,44100)
test.show()
test.play_audio()